In [2]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [6]:
query = 'SELECT ano,sigla_uf,  id_municipio, sum(alunos_soma) as alunos_soma, sum(professores_soma) as professores_soma FROM `repositoriodedadosgpsp.Datalake.INEP_media_aluno_docente` where rede!="privada" group by ano,sigla_uf,  id_municipio order by ano'

In [3]:
client = bigquery.Client()

In [7]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 66306/66306 [00:05<00:00, 12910.86rows/s]


In [5]:
df

,ano,sigla_uf,id_municipio,alunos_soma,professores_soma
0,2009,DF,5300108,1762488,67280
1,2009,AP,1600808,12373,654
2,2009,AC,1200427,8514,417
3,2009,RR,1400100,271917,12899
4,2009,AC,1200203,70364,3159
...,...,...,...,...,...
66301,2020,TO,1718659,1185,68
66302,2020,TO,1715507,930,61
66303,2020,RS,4305850,1180,158
66304,2020,SC,4215059,1424,128


In [10]:
sum(df['professores_soma'])

83163369

In [7]:
## antigo

sum(df['professores_soma'])

83849604

In [38]:
## Subindo para o GBQ

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66306 entries, 0 to 66305
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ano               66306 non-null  Int64 
 1   sigla_uf          66306 non-null  object
 2   id_municipio      66306 non-null  object
 3   alunos_soma       66306 non-null  Int64 
 4   professores_soma  66306 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 2.7+ MB


In [4]:


schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('id_municipio','STRING',description='ID do município no IBGE'),
 bigquery.SchemaField('alunos_soma','INTEGER',description='Soma de aluno'),
 bigquery.SchemaField('professores_soma','INTEGER',description='Soma de aluno')
 ]

In [8]:
dataset_ref = client.dataset('perfil_remuneracao')
table_ref = dataset_ref.table('INEP_media_aluno_docente_ano')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=59ddbc5b-a5d3-4aec-a011-c7ce7c11c57d>